# GPIO Introduction

First order of business is to import some libraries to help send/receive signals to/from the GPIO pins of the Raspberry Pi.

In [ ]:
import RPi.GPIO as GPIO
import time

The following code will set the mode to a Broadcom pin layout (which seems to be what we want), and set up our GPIO pin 25 as an output pin.

In [ ]:
outpin = 25
GPIO.setmode(GPIO.BCM)
GPIO.setup(outpin, GPIO.OUT)

In [ ]:
GPIO.output(outpin, True) # hello world?

In [ ]:
GPIO.output(outpin, False) # goodbye world?

## SOS example

In [ ]:
timeunit  = 0.08
dashtime  = timeunit*3.0
dottime   = timeunit
dotgap    = timeunit
lettergap = timeunit*2.0
wordgap   = timeunit*7.0

In [ ]:
GPIO.output(outpin, False) # start with the LED off
time.sleep(1.0)

for i in range(0,2):  # SOS, the hard way...
    GPIO.output(outpin, True)
    time.sleep(dottime)
    GPIO.output(outpin, False)
    time.sleep(dotgap)
    GPIO.output(outpin, True)
    time.sleep(dottime)
    GPIO.output(outpin, False)
    time.sleep(dotgap)
    GPIO.output(outpin, True)
    time.sleep(dottime)
    GPIO.output(outpin, False)
    time.sleep(lettergap)
    GPIO.output(outpin, True)
    time.sleep(dashtime)
    GPIO.output(outpin, False)
    time.sleep(dotgap)
    GPIO.output(outpin, True)
    time.sleep(dashtime)
    GPIO.output(outpin, False)
    time.sleep(dotgap)
    GPIO.output(outpin, True)
    time.sleep(dashtime)
    GPIO.output(outpin, False)
    time.sleep(lettergap)
    GPIO.output(outpin, True)
    time.sleep(dottime)
    GPIO.output(outpin, False)
    time.sleep(dotgap)
    GPIO.output(outpin, True)
    time.sleep(dottime)
    GPIO.output(outpin, False)
    time.sleep(dotgap)
    GPIO.output(outpin, True)
    time.sleep(dottime)
    GPIO.output(outpin, False)
    time.sleep(wordgap)

In [ ]:
morsedict = {
        'A': '.-',
        'B': '-...',
        'C': '-.-.',
        'D': '-..',
        'E': '.',
        'F': '..-.',
        'G': '--.',
        'H': '....',
        'I': '..',
        'J': '.---',
        'K': '-.-',
        'L': '.-..',
        'M': '--',
        'N': '-.',
        'O': '---',
        'P': '.--.',
        'Q': '--.-',
        'R': '.-.',
        'S': '...',
        'T': '-',
        'U': '..-',
        'V': '...-',
        'W': '.--',
        'X': '-..-',
        'Y': '-.--',
        'Z': '--..',
        '0': '-----',
        '1': '.----',
        '2': '..---',
        '3': '...--',
        '4': '....-',
        '5': '.....',
        '6': '-....',
        '7': '--...',
        '8': '---..',
        '9': '----.',           
        ' ': ' ',
        ',': '--..--',
        '.': '.-.-.-',
        '?': '..--..',
        ';': '-.-.-.',
        ':': '---...',
        "'": '.----.',
        '-': '-....-',
        '/': '-..-.',
        '(': '-.--.-',
        ')': '-.--.-',
        '_': '..--.-',  
    }

dotdashmap = {".": dottime, "-": dashtime, " ": wordgap}

In [ ]:
def blink_letter(letter):
    for s in morsedict[letter.upper()]:
        GPIO.output(outpin, True)
        time.sleep(dotdashmap[s])
        GPIO.output(outpin, False)
        time.sleep(dotgap)
    time.sleep(lettergap)  # end of letter time gap (for a total of 3x the dot unit.)

Let's break this down a bit before we see what happens when we call this function.

In [ ]:
s = "q"

In [ ]:
s.upper()

In [ ]:
morsedict['O']

In [ ]:
dotdashmap['-']

In [ ]:
blink_letter('o')

In [ ]:
def blink_message(message):
    for letter in message:
        blink_letter(letter)

In [ ]:
blink_message("World.")

## Buttons

Buttons require us to 'listen' indefinitely for input to our GPIO pin.  We use an infinite loop to do this.  **To stop the loop, you simply click Kernel/Interrupt in the menu above.**

Once the button is wired to the pin designated below (inpin #) with the appropriate resistors, set up the port as an input port:

In [ ]:
inpin = 20
GPIO.setup(inpin, GPIO.IN, pull_up_down=GPIO.PUD_UP) # the pull_up_down keeps us from worrying about pull-up resistors

In [ ]:
prev_input = 1 # looking for a 'False' as the button press signal, so we start with a "True"
while True:
    #take a reading
    input = GPIO.input(inpin)
    #if the last reading was low and this one high, print
    if ((not prev_input) and input):
        print("Button pressed")
    #update previous input
    prev_input = input
    #slight pause to debounce
    time.sleep(0.05)

So, now let's do something with that button press -- how about flashing our light, then taking a photo.

We'll first import the camera class...

In [ ]:
from picamera import PiCamera, Color
from time import sleep

We'll wait to instantiate our camera object, just so we don't crash anything with a bunch of button presses.

In [ ]:
def blink_and_shoot(idx):
    """ idx: an index number for photos to be stored at /img/snapshot-[idx] """
    
    # set up camera
    camera = PiCamera()
    #camera.resolution = (480, 320)  # uncomment this line if you want smaller photos
    camera.vflip = True
    camera.hflip = True
    camera.start_preview()
    
    blink_message("s")

    camera.image_effect = 'oilpaint' # just to make things interesting
    sleep(0.5)
    camera.capture('img/snapshot-%s.jpg' % idx)
    camera.stop_preview()
    camera.close()

Now for the infinite button loop to listen for a press:

In [ ]:
prev_input = 1 # looking for a 'False' as the button press signal, so we start with a "True"
counter = 1
while True:
    #take a reading
    input = GPIO.input(inpin)
    #if the last reading was low and this one high, print
    if ((not prev_input) and input):
        print("Taking photo%s" % counter)
        blink_and_shoot(counter)
        counter += 1
    #update previous input
    prev_input = input
    #slight pause to debounce
    time.sleep(0.05)